In [1]:
using LinearAlgebra, BenchmarkTools

In [5]:
"""
    Ls(A, b)

하삼각행렬 L 에 대해 Lx=b 의 해 x 를 구한다.
"""
function Ls(L::Matrix{T}, b::Union{Vector{T}, Matrix{T}}) where T<:Number
    M, N = size(L)    
    @assert M == N == size(b)[1]
    x = zeros(T, N)
    x[1] = b[1]/L[1, 1]
    for i in 2:N
        x[i] = b[i]
        for j in 1:1:(i-1)
            x[i] -= L[i, j]*x[j]
        end
        x[i] = x[i]/L[i, i]
    end
    return x
end


"""
    Us(A, b)

상삼각행렬 U 에 대해 Ux=b 의 해 x 를 구한다.
"""
function Us(U::Matrix{T},b::Union{Vector{T}, Matrix{T}}) where T<:Number
    M, N = size(U)    
    @assert M == N
    x = zeros(T, N)
    x[N] = b[N]/U[N, N]
    for i in (N-1):-1:1
        x[i] = b[i]
        for j in (i+1):1:N
            x[i] -= U[i, j] * x[j]
        end
        x[i] = x[i]/U[i, i]
    end
    return x
end


Us

In [25]:
function gauss_jordan_elimination(A::Matrix{T}, b::Union{Nothing, Vector{T}, Matrix{T}}=nothing; eptols = 1.0e-10) where T<:Real
    M, N = size(A)

    if b ≠ nothing
        @assert M == size(b)[1]
        B = [A b]
    else
        B = A
    end
    
    ld = 0 #선행 1 성분의 column index

    for i in 1:M
        termination = true # 종료 조건
        for j in (ld+1):N
            p = argmax(abs.(B[i:end, j])) + i -1
            
            if abs(B[p, j]) > eptols
                B[i,:], B[p, :] = B[p, :], B[i,:]
                ld = j
                termination = false
                break
            end
        end

        if termination 
            break
        end
        
        B[i, :] .= B[i, :]./B[i, ld]
        
        # 선행 1 성분의 열을 자신을 제외하고는 제거
        for k in 1:M
            if k ≠ i 
                B[k, :] .= B[k, :] .- (B[k, ld].*B[i, :])
            end
        end
    end

    if b ≠ nothing 
         return B[:, 1:N], B[:,(N+1):end]
    else 
         return B
    end
end

gauss_jordan_elimination (generic function with 2 methods)

In [26]:
A, b = Float64.([2 1 1;0 1 0;3 -2 3]), [1., 3, 2]
A1, b1 = gauss_jordan_elimination(A, b)

([1.0 0.0 0.0; 0.0 1.0 0.0; 0.0 0.0 1.0], [-4.666666666666667; 3.0; 7.333333333333333;;])

In [27]:
A1, A2=gauss_jordan_elimination(A, one(A))

([1.0 0.0 0.0; 0.0 1.0 0.0; 0.0 0.0 1.0], [1.0 -1.6666666666666665 -0.3333333333333335; 0.0 1.0 0.0; -1.0 2.333333333333333 0.6666666666666667])

In [28]:
A2*A

3×3 Matrix{Float64}:
 1.0   4.44089e-16  -4.44089e-16
 0.0   1.0           0.0
 0.0  -4.44089e-16   1.0

In [33]:
function gauss0(A::Matrix{T}, b::Union{Nothing, Vector{T}, Matrix{T}}=nothing; eptols = 1.0e-10) where T<:Real
    M, N = size(A)

    if b ≠ nothing
        @assert M == size(b)[1]
        U = [A b]
    else
        U = copy(A)
    end

    ld = 0 #선행 1 성분의 column index

    for i in 1:M
        termination = true # 종료 조건
        for j in (ld+1):N
            p = argmax(abs.(U[i:end, j])) + i -1
            
            if abs(U[p, j]) > eptols
                U[i,:], U[p, :] = U[p, :], U[i,:]
                ld = j
                termination = false
                break
            end
        end

        if termination 
            break
        end
        
        # 선행 성분의 열을 자신을 제외하고는 제거
        for k in (i+1):M
            if k ≠ i 
                U[k, :] .= U[k, :] .- (U[k, ld].*U[i, :])./U[i, ld]
            end
        end
    end

    if b ≠ nothing 
        return U[:, 1:N], U[:,(N+1):end]
   else 
        return U
   end
end

gauss0 (generic function with 2 methods)

In [36]:
 A3, b3=gauss0(A, b)

([3.0 -2.0 3.0; 0.0 2.333333333333333 -1.0; 0.0 0.0 0.4285714285714286], [2.0; -0.33333333333333326; 3.142857142857143;;])

In [42]:
xx = Us(A3, b3)

3-element Vector{Float64}:
 -4.666666666666667
  3.0000000000000004
  7.333333333333333

In [44]:
A*xx

3-element Vector{Float64}:
 0.9999999999999991
 3.0000000000000004
 2.0

In [110]:
function PLU(A::Matrix{T}; eptols = 1.0e-10) where T<:Real
    M, N = size(A)
    @assert M == N

    L, P, U = one(A), one(A), one(A)
    B = copy(A)
    
    for i in 1:(M)
        p = argmax(abs.(B[i:end, i])) + i -1
        
        if abs(B[p, i]) < eptols
            error("Singularity error")    
        end

        P[i,:], P[p, :] = P[p, :], P[i, :]
        B[i,:], B[p, :] = B[p, :], B[i, :]
        
        U[i, i] = B[i, i]
        U[i, i+1:end] .= B[i, i+1:end]
        L[i+1:end, i] .= B[i+1:end, i] ./ U[i, i]
        B[i+1:end, i+1:end] .= B[i+1:end, i+1:end] - (L[i+1:end, i:i] * U[i:i, i+1:end])
    end
    return L, U, P
end

PLU (generic function with 1 method)

In [125]:
CC=rand(Float64, (12, 12)) + 3*I

12×12 Matrix{Float64}:
 3.97335    0.0109225  0.547702   0.649882   …  0.188207  0.177987   0.554972
 0.977061   3.75282    0.151345   0.728188      0.126441  0.696274   0.346121
 0.026027   0.994871   3.89925    0.990191      0.387402  0.350711   0.695096
 0.271917   0.609679   0.59676    3.62555       0.727354  0.838565   0.799854
 0.063271   0.499874   0.575026   0.365036      0.126075  0.0607189  0.41274
 0.771585   0.716798   0.144201   0.14546    …  0.461099  0.0863546  0.221212
 0.108126   0.0960933  0.803818   0.383693      0.418134  0.38503    0.847912
 0.98581    0.379122   0.0398761  0.391686      0.957993  0.339588   0.842759
 0.0169151  0.678634   0.396327   0.735744      0.183008  0.694117   0.135908
 0.876935   0.0956017  0.669662   0.337535      3.74942   0.959084   0.630445
 0.537954   0.726942   0.574854   0.0604795  …  0.962293  3.12479    0.693873
 0.66475    0.26575    0.171625   0.619644      0.7966    0.0675967  3.01487

In [126]:
L1, U1, P1=PLU(CC)

([1.0 0.0 … 0.0 0.0; 0.24590349361647845 1.0 … 0.0 0.0; … ; 0.13539056206509953 0.19344968970961185 … 1.0 0.0; 0.16730197455558576 0.07037671152300816 … -0.07557964017944593 1.0], [3.9733513095805613 0.010922527983854957 … 0.17798724827264611 0.5549721536640151; 0.0 3.750137222346635 … 0.65250646253448 0.2096515912819225; … ; 0.0 0.0 … 2.7641837997770304 0.23419259750740537; 0.0 0.0 … 0.0 2.623196491440411], [1.0 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.0 0.0 … 1.0 0.0; 0.0 0.0 … 0.0 1.0])

In [129]:
maximum(abs.(P1*CC - L1*U1))

8.881784197001252e-16

In [122]:
L1

5×5 Matrix{Float64}:
 1.0        0.0        0.0          0.0        0.0
 0.0744654  1.0        0.0          0.0        0.0
 0.0511688  0.285629   1.0          0.0        0.0
 0.0388027  0.18208   -0.00844316   1.0        0.0
 0.0722443  0.270631  -0.0354912   -0.0608272  1.0

In [123]:
U1

5×5 Matrix{Float64}:
 3.68344  0.293117  0.78976   0.312552  0.178957
 0.0      3.27244   0.620451  0.824096  0.0827883
 0.0      0.0       3.33487   0.601478  0.396173
 0.0      0.0       0.0       3.134     0.70968
 0.0      0.0       0.0       0.0       3.0551

In [124]:
lu(CC)

LU{Float64, Matrix{Float64}, Vector{Int64}}
L factor:
5×5 Matrix{Float64}:
 1.0        0.0        0.0          0.0        0.0
 0.0744654  1.0        0.0          0.0        0.0
 0.0511688  0.285629   1.0          0.0        0.0
 0.0388027  0.18208   -0.00844316   1.0        0.0
 0.0722443  0.270631  -0.0354912   -0.0608272  1.0
U factor:
5×5 Matrix{Float64}:
 3.68344  0.293117  0.78976   0.312552  0.178957
 0.0      3.27244   0.620451  0.824096  0.0827883
 0.0      0.0       3.33487   0.601478  0.396173
 0.0      0.0       0.0       3.134     0.70968
 0.0      0.0       0.0       0.0       3.0551